<a href="https://colab.research.google.com/github/brianp0513/NLP-project/blob/main/LSTM_NLP_Project_version5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# model 1: two bidirectional LSTM
# model 2: one bidirectional LSTM


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from keras.models import Model
from keras.layers import LSTM, Activation, Dense, Dropout, Input, Embedding
from keras.optimizers import RMSprop
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
from keras.utils import to_categorical
from keras.callbacks import EarlyStopping
import tensorflow as tf
%matplotlib inline
import nltk
from nltk.stem import PorterStemmer,WordNetLemmatizer
# import seaborn as sns
import time
import re
from wordcloud import WordCloud
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
import random
import os
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
np.random.seed(1234)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [3]:
def getTextFromFiles(df, data_path, depression, limit):
    """Return Data Frame """

    for file in os.listdir(data_path)[:limit]:
        with open(data_path + "/" + file, 'r', encoding="ISO-8859-1") as file1:
            file1 = file1.read()
            df = df.append({'text': file1, 'depression': int(depression)}, ignore_index=True)

    return df

In [40]:
data_path_d = "/content/drive/My Drive/NLP Team/code/kerasData/reddit_depression"
data_path_nd = "/content/drive/My Drive/NLP Team/code/kerasData/reddit_non_depression"
df = pd.DataFrame(columns=['text', 'depression'])
df = getTextFromFiles(df, data_path_d, 1, 500)
df = getTextFromFiles(df, data_path_nd, 0, 500)


In [220]:
X = df['text'].to_numpy()
Y = df['depression'].to_numpy()
X_train,X_test,Y_train,Y_test = train_test_split(X,Y,test_size=0.15)
# max_words = 1000
max_words = 1000
max_len = 300

tokenizer = Tokenizer(num_words=max_words)

tokenizer.fit_on_texts(X_train)

sequences = tokenizer.texts_to_sequences(X_train)
# sequences_matrix = tokenizer.sequences_to_matrix(sequences, mode='binary')
# sequences_matrix = tokenizer.sequences_to_matrix(sequences, mode='count')
# sequences_matrix = tokenizer.sequences_to_matrix(sequences, mode='freq')
sequences_matrix = sequence.pad_sequences(sequences,maxlen=max_len)

test_sequences = tokenizer.texts_to_sequences(X_test)

# test_sequences_matrix = tokenizer.sequences_to_matrix(test_sequences, mode='binary')
# test_sequences_matrix = tokenizer.sequences_to_matrix(test_sequences, mode='count')
# test_sequences_matrix = tokenizer.sequences_to_matrix(test_sequences, mode='freq')
test_sequences_matrix = sequence.pad_sequences(test_sequences,maxlen=max_len)


In [ ]:
print(X_train.shape)
print(np.array(sequences).shape)
# print(np.array(sequences))
print(sequences_matrix.shape)
# print(Y_train.shape)
# print(tok.word_index)

In [251]:

model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Embedding(input_dim = max_words, output_dim = 8, input_length= max_len))
model.add(tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64 , return_sequences=True)))
model.add(tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64)))
model.add(tf.keras.layers.Dropout(0.2))
model.add(tf.keras.layers.Dense(80))
model.add(tf.keras.layers.Activation('relu'))
model.add(tf.keras.layers.Dropout(0.2))
model.add(tf.keras.layers.Dense(1))
model.add(tf.keras.layers.Activation('sigmoid'))
model.build(input_shape=sequences_matrix.shape)
opt = tf.keras.optimizers.Adam(learning_rate=0.0001)
model.compile(loss='binary_crossentropy',optimizer=opt,metrics=['accuracy'])

In [ ]:

model.fit(sequences_matrix, Y_train.astype(float), batch_size=4,epochs=40,
          validation_split=0.2,callbacks=[EarlyStopping(monitor='val_loss',min_delta=0.0001)])
accr = model.evaluate(test_sequences_matrix,Y_test.astype(float))
print("1 bidirectional LSTM")
print('Test set\n  Loss: {:0.3f}\n  Accuracy: {:0.3f}'.format(accr[0],accr[1]))

In [328]:
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Embedding(input_dim = max_words, output_dim = 10, input_length= max_len))


model.add(tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(20, return_sequences=True)))
model.add(tf.keras.layers.Dense(80))
model.add(tf.keras.layers.Activation('tanh'))
model.add(tf.keras.layers.Dropout(0.2))
model.add(tf.keras.layers.Dense(1))
model.add(tf.keras.layers.Activation('sigmoid'))
model.build(input_shape=sequences_matrix.shape)
# opt = tf.keras.optimizers.RMSprop(learning_rate=0.0001)
# model.compile(loss='binary_crossentropy',optimizer=opt,metrics=['accuracy'])
opt = tf.keras.optimizers.Adam(learning_rate=0.0003)
model.compile(loss='binary_crossentropy',optimizer=opt,metrics=['accuracy'])
model.summary()

In [330]:
model.fit(sequences_matrix, Y_train.astype(float), batch_size=4,epochs=40,
          validation_split=0.2,callbacks=[EarlyStopping(monitor='val_loss',min_delta=0.0001)])
accr = model.evaluate(test_sequences_matrix,Y_test.astype(float))
print("1 bidirectional LSTM")
print('Test set\n  Loss: {:0.3f}\n  Accuracy: {:0.3f}'.format(accr[0],accr[1]))

Epoch 1/40
170/170 [==============================] - 4s 25ms/step - loss: 0.6885 - accuracy: 0.5701 - val_loss: 0.6732 - val_accuracy: 0.5915
Epoch 2/40
170/170 [==============================] - 4s 21ms/step - loss: 0.6607 - accuracy: 0.6102 - val_loss: 0.6261 - val_accuracy: 0.6639
Epoch 3/40
170/170 [==============================] - 4s 21ms/step - loss: 0.5543 - accuracy: 0.7405 - val_loss: 0.4837 - val_accuracy: 0.8342
Epoch 4/40
170/170 [==============================] - 4s 22ms/step - loss: 0.4004 - accuracy: 0.8590 - val_loss: 0.4515 - val_accuracy: 0.8533
Epoch 5/40
170/170 [==============================] - 3s 20ms/step - loss: 0.3360 - accuracy: 0.8826 - val_loss: 0.4424 - val_accuracy: 0.8644
Epoch 6/40
170/170 [==============================] - 4s 21ms/step - loss: 0.3121 - accuracy: 0.8928 - val_loss: 0.4228 - val_accuracy: 0.8584
Epoch 7/40
170/170 [==============================] - 4s 21ms/step - loss: 0.2592 - accuracy: 0.9197 - val_loss: 0.4341 - val_accuracy: 0.8632

5/5 [==============================] - 0s 13ms/step - loss: 0.4467 - accuracy: 0.8472


Test set
  Loss: 0.447
  Accuracy: 0.847
